#Assignments Week 2 - Key

## Assignment 1: Extract n-grams
Write a function "ngrams", which retrieves the list of n-grams for a given input sentence, for any value of n.

In [ ]:
import nltk
nltk.download('punkt')

sentence = "this is a test sentence." 

def ngrams(sentence, n):
    ngrams_list = []
    
    tokens = nltk.word_tokenize(sentence)
    tokens = ["<s>"]*(n-1) + tokens + ["</s>"]
    
    for i in range(0, len(tokens)-n+1):
        ngram = tokens[i:i+n]
        ngrams_list.append(ngram)
    return ngrams_list

print("Bigrams:")
print(ngrams(sentence, 2))
print("Trigrams:")
print(ngrams(sentence, 3))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Bigrams:
[['<s>', 'this'], ['this', 'is'], ['is', 'a'], ['a', 'test'], ['test', 'sentence'], ['sentence', '.'], ['.', '</s>']]
Trigrams:
[['<s>', '<s>', 'this'], ['<s>', 'this', 'is'], ['this', 'is', 'a'], ['is', 'a', 'test'], ['a', 'test', 'sentence'], ['test', 'sentence', '.'], ['sentence', '.', '</s>'], ['.', '</s>', '</s>']]


## Assignment 2 - Predict the next word 
Write a function "predict_next_word" to get phrase (of at least two words long) from the user as input and predict the most probably next word by analyzing 3-grams in the 3-gram dictionary we created, in which the word is the last word (i.e. we look at the last two words or the 2-gram to predict the 3rd word).

If there are no valid options (no probable words can be found), print an error message.
Print the suggested word and the probability of the trigram ending with it.

Example input: __this is__ <br>
Output:<br>
__a (prob. 0.1428)__<br>
<br>
Example input: __i like__ <br>
Output:<br>
__him (prob. 0.2727)__<br>
<br>
Example input: __i like to eat__ <br>
Output:<br>
__go (prob. 0.1086)__ (this prob is calculated by only looking at the two previous words "to eat".<br>
<br>
Example input: __when sometimes tomorrow__ <br>
Output:<br>
__No words could be predicted!__<br>


In [ ]:
import json
from operator import itemgetter

with open('./3gram_dict.json', 'r') as fp:
    emma_word_prob_3 = json.load(fp)

def predict_next_word():
    phrase = input("Please provide an unfinished sentence of at least two words long: ")
    # First we get the tokens
    tokens = nltk.word_tokenize(phrase)

    # and the get the last two words
    bigram = tokens[-2:]
    #print(bigram)

    # We can collect all 3-grams in our dictionary starting with these two words to a list
    candidates = []
    for key, value in emma_word_prob_3.items():
        # We need to merge the two-gram to a single string so that we can match them as keys in the dictionary
        if key.startswith(" ".join(bigram)):
            # If we find a match, add the key to the candidates list
            candidates.append([key, value])

    # If there are no candidates, print a message
    if len(candidates) == 0:
        print("No words could be predicted!")

 
    candidates_sorted  = sorted(candidates, key=itemgetter(1), reverse=True) # sort the list of lists by probability
    
    word = candidates_sorted[0][0].split()[-1]
    prob = candidates_sorted[0][1]
    print(word, "(prob. "+ str(prob) + ")")
    


predict_next_word()

Please provide an unfinished sentence of at least two words long: this is
a (prob. 0.14285714285714285)


## Assignment 3 - Predict the next word continued
Let's modify the code above to print 5 most probably words as the next word.


In [ ]:
from operator import itemgetter

def predict_next_word():
    phrase = input("Please provide an unfinished sentence of at least two words long: ")
    # First we get the tokens
    tokens = nltk.word_tokenize(phrase)

    # and the get the last two words
    bigram = tokens[-2:]
    #print(bigram)

    # We can collect all 3-grams in our dictionary starting with these two words to a list
    candidates = []
    for key, value in emma_word_prob_3.items():
        # We need to merge the two-gram to a single string so that we can match them as keys in the dictionary
        if key.startswith(" ".join(bigram)):
            # If we find a match, add the key to the candidates list
            candidates.append([key, value])

    # If there are no candidates, print a message
    if len(candidates) == 0:
        print("No words could be predicted!")

 
    candidates_sorted  = sorted(candidates, key=itemgetter(1), reverse=True) # sort the list of lists by probability
    print(candidates_sorted)
    for i in range(5):
        word = candidates_sorted[i][0].split()[-1]
        prob = candidates_sorted[i][1]
        print(word, "(prob. "+ str(prob) + ")")
    


predict_next_word()

Please provide an unfinished sentence of at least two words long: this is
[['this is a', 0.14285714285714285], ['this is quite', 0.08928571428571429], ['this is the', 0.08928571428571429], ['this is an', 0.05357142857142857], ['this is not', 0.05357142857142857], ['this is what', 0.05357142857142857], ['this is all', 0.03571428571428571], ['this is brilliant', 0.03571428571428571], ['this is just', 0.03571428571428571], ['this is most', 0.03571428571428571], ['this is very', 0.03571428571428571], ['this is admirable', 0.017857142857142856], ['this is by', 0.017857142857142856], ['this is charming', 0.017857142857142856], ['this is coming', 0.017857142857142856], ['this is delightful', 0.017857142857142856], ['this is feeling', 0.017857142857142856], ['this is from', 0.017857142857142856], ['this is great', 0.017857142857142856], ['this is impossible', 0.017857142857142856], ['this is in', 0.017857142857142856], ['this is like', 0.017857142857142856], ['this is living', 0.01785714285714

## Assignment 4: Text Generation 
Let's generate some "good" sentences of 10 words long by using random ngrams from a given ngram dictionary.

- Start with a random n-gram from the dictionary
- Add words to sentence by finding probable words that follow the previous n-1 grams, until 10 words are generated.
- if it is not possible to find a probable next word at some point, print the sentence generated so far and print a message "No other words could be added.".




In [ ]:

import random
import json
import sys

with open('./3gram_dict.json', 'r') as fp:
    emma_word_prob_3 = json.load(fp)


def make_random_sentence(prob_dict, n):
    sentence = []
    for i in range(10):
        #print(i)
        if i == 1:
            #print("First n-gram:")
            ngram = random.choice(list(prob_dict))
            tokens = ngram.split()
            #print(ngram)
            for token in tokens:
                sentence.append(token)
        else:
            
            #print(sentence)
            start = sentence[-n+1:] # get the last n-1 tokens to find a new ngram that starts with it
            #print(start)
            candidates = []
            for key in prob_dict:
                if key.startswith(" ".join(start)):
                    candidates.append(key)
            try:
                ngram = random.choice(candidates)
                #print(ngram)
                word = ngram.split()[-1]
                sentence.append(word)
                #print(sentence)
            except:
                text = " ".join(sentence)
                print(text)
                print("No other words could be added.")
                sys.exit()
            
       

    text = " ".join(sentence)
    print(text)

make_random_sentence(emma_word_prob_3, 3)

her : you will stay , yes -- it _is_ possible --"


## Assignment 5 - Text generation continued
Adapt the code above so that:
- it starts with a random n-gram that begins with "(s)"
- it restarts generating a sentence if no valid ngrams are found during generation of a sentence of 10 words.
    
Tip: Which one is better: a for loop or a while loop. Why?

In [ ]:
import random

def make_random_sentence(prob_dict, n):

    sentence = []
    i = 0
    while i < 10:
        i+=1
        #print(i)
        if i == 1:
            start_candidates = []
            for key, value in prob_dict.items():
                if key.startswith("<s> <s>"):
                    start_candidates.append(key)
            ngram = random.choice(start_candidates)
            tokens = ngram.split()
            for token in tokens:
                sentence.append(token)
        else:
            try:
                #print(sentence)
                start = sentence[-n+1:] # get the last n-1 tokens to find a new ngram that starts with it
                #print(start)
                candidates = []
                for key in prob_dict:
                    if key.startswith(" ".join(start)):
                        candidates.append(key)
                ngram = random.choice(candidates)
                #print(ngram)
                word = ngram.split()[-1]
                sentence.append(word)
            except:
                print("resetting ... ")
                print(" ".join(sentence))
                i = 0
                sentence = []
       

    text = " ".join(sentence)
    print(text)

make_random_sentence(emma_word_prob_3, 3)

<s> <s> randalls is doubly due , and throw up a she
